<h4><center><center>WEBSCRAPEO DE ACTIVIDADES POR DEPARTAMENTO</center></h4>
<center><img src="https://cdn.ventrata.com/image/upload/s--4sBy-e_C--/c_fill,dpr_3.0,w_200/v1714038213/getyourguideticketing_c5gntp.png?_a=BACCg+GT" alt="Alojamiento en Airbnb" width="300"/>
</div></center>

In [1]:
!pip install selenium webdriver-manager beautifulsoup4

In [2]:
!pip install pandas

In [3]:
# Lista de departamentos del Perú
departamentos = [
    "Amazonas", "Áncash", "Apurímac", "Arequipa", "Ayacucho", "Cajamarca",
    "Callao", "Cusco", "Huancavelica", "Huánuco", "Ica", "Junín", "La Libertad",
    "Lambayeque", "Lima", "Loreto", "Madre de Dios", "Moquegua", "Pasco",
    "Piura", "Puno", "San Martín", "Tacna", "Tumbes", "Ucayali"
]

# Limpiar los nombres para la URL (reemplazar espacios por +, acentos por caracteres simples)
import urllib.parse

# Lista donde se almacenarán los resultados
links = []

# Recorrer cada departamento para crear su enlace personalizado
for depto in departamentos:
    # Codificar el nombre del departamento para URL
    query = urllib.parse.quote(depto)
    url = f"https://www.getyourguide.es/s/?q={query}&searchSource=3"
    links.append({"Departamento": depto, "Link": url})  # Agregar el resultado a la lista como un diccionario

# Mostrar resultados
import pandas as pd
df = pd.DataFrame(links) # Crear el DataFrame con las columnas 'Departamento' y 'Link'
df.head(24)

,Departamento,Link
0,Amazonas,https://www.getyourguide.es/s/?q=Amazonas&sear...
1,Áncash,https://www.getyourguide.es/s/?q=%C3%81ncash&s...
2,Apurímac,https://www.getyourguide.es/s/?q=Apur%C3%ADmac...
3,Arequipa,https://www.getyourguide.es/s/?q=Arequipa&sear...
4,Ayacucho,https://www.getyourguide.es/s/?q=Ayacucho&sear...
5,Cajamarca,https://www.getyourguide.es/s/?q=Cajamarca&sea...
6,Callao,https://www.getyourguide.es/s/?q=Callao&search...
7,Cusco,https://www.getyourguide.es/s/?q=Cusco&searchS...
8,Huancavelica,https://www.getyourguide.es/s/?q=Huancavelica&...
9,Huánuco,https://www.getyourguide.es/s/?q=Hu%C3%A1nuco&...


In [4]:
# Librerías necesarias
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import time

# Lista de departamentos del Perú para consultar en la web
departamentos = [
    "Amazonas", "Áncash", "Apurímac", "Arequipa", "Ayacucho", "Cajamarca",
    "Callao", "Cusco", "Huancavelica", "Huánuco", "Ica", "Junín", "La Libertad",
    "Lambayeque", "Lima", "Loreto", "Madre de Dios", "Moquegua", "Pasco",
    "Piura", "Puno", "San Martín", "Tacna", "Tumbes", "Ucayali"
]

# Configuración del navegador
options = Options()
# options.add_argument("--headless")  # Descomenta para que no abra el navegador
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# Iniciar el navegador con configuración automática del driver

# Lista para almacenar la información recolectada de cada actividad turística
actividades = []

# Bucle principal: recorre cada departamento y extrae actividades turísticas
for depto in departamentos:
    try:
        print(f"Procesando: {depto}")
        query = urllib.parse.quote(depto)
        url = f"https://www.getyourguide.es/s/?q={query}&searchSource=3"
        driver.get(url)

        # Espera a que se cargue el contenido
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a[data-test-id="vertical-activity-card-link"]'))
        )
         # Obtener contenido HTML de la página
        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.select('a[data-test-id="vertical-activity-card-link"]')

        # Extraer datos de cada tarjeta de actividad
        for card in cards:
            title_tag = card.select_one('h3')
            title = title_tag.get_text(strip=True) if title_tag else None

            duration_tag = card.select_one('li.activity-attributes__attribute')
            duration = duration_tag.get_text(strip=True) if duration_tag else None

            rating_tag = card.select_one('div.c-activity-rating__rating')
            rating = rating_tag.get_text(strip=True).replace(',', '.') if rating_tag else None

            reviews_tag = card.select_one('div.c-activity-rating__label')
            reviews = reviews_tag.get_text(strip=True) if reviews_tag else None

            price_tag = card.select_one('span.activity-price__text-price')
            price = price_tag.get_text(strip=True).replace('\xa0', ' ') if price_tag else None

            href = card.get("href")
            full_url = "https://www.getyourguide.es" + href if href else None


            # Agregar los datos extraídos a la lista
            actividades.append({
                "Departamento": depto,
                "Actividad": title,
                "Duración": duration,
                "Precio": price,
                "Calificación": rating,
                "Reseñas": reviews,
                "Enlace": full_url
            })

        # Pequeña pausa para evitar ser bloqueado por la web
        time.sleep(1)  

    except Exception as e:
        print(f"Error en {depto}: {e}")
        continue

# Cierra el navegador
driver.quit()

# Convertir a DataFrame
df = pd.DataFrame(actividades)
df.head()

Procesando: Amazonas
Procesando: Áncash
Error en Áncash: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=138.0.7204.101)
Stacktrace:
	GetHandleVerifier [0x0x10c44a3+62419]
	GetHandleVerifier [0x0x10c44e4+62484]
	(No symbol) [0x0xf02133]
	(No symbol) [0x0xee0fd9]
	(No symbol) [0x0xf75b9e]
	(No symbol) [0x0xf90159]
	(No symbol) [0x0xf6f266]
	(No symbol) [0x0xf3e852]
	(No symbol) [0x0xf3f6f4]
	GetHandleVerifier [0x0x1334793+2619075]
	GetHandleVerifier [0x0x132fbaa+2599642]
	GetHandleVerifier [0x0x10eb04a+221050]
	GetHandleVerifier [0x0x10db2c8+156152]
	GetHandleVerifier [0x0x10e1c7d+183213]
	GetHandleVerifier [0x0x10cc388+94904]
	GetHandleVerifier [0x0x10cc512+95298]
	GetHandleVerifier [0x0x10b766a+9626]
	BaseThreadInitThunk [0x0x764a5d49+25]
	RtlInitializeExceptionChain [0x0x771dd1ab+107]
	RtlGetAppContainerNamedObjectPath [0x0x771dd131+561]

Procesando: Apurímac
Error en Apurímac: Message: no such window: target window

,Departamento,Actividad,Duración,Precio,Calificación,Reseñas,Enlace
0,Amazonas,"Manaos: Tour de 2, 3 o 4 días por la selva ama...",2 - 4 días • Grupo privado • Recogida disponible,200 USD,3.9,(613),https://www.getyourguide.es/manaos-l917/manaos...
1,Amazonas,Selva Amazónica: Excursión de 3 ó 4 días en Ip...,2 - 4 días • Grupo reducido • Recogida disponible,180 USD,4.1,(180),https://www.getyourguide.es/manaos-l917/excurs...
2,Amazonas,Iquitos: aventura de 2 días en la selva amazón...,2 días • Grupo reducido,347 USD,3.7,None,https://www.getyourguide.es/iquitos-l32156/iqu...
3,Amazonas,"Manaos: tour de 2, 3 o 4 días",2 - 4 días • Grupo reducido,220 USD,4.8,(33),https://www.getyourguide.es/manaos-l917/selva-...
4,Amazonas,Manaos: Tour de día completo por el río Amazonas,1 día • Recogida disponible,55 USD,4.1,(288),https://www.getyourguide.es/manaos-l917/tour-d...


In [5]:
df.to_csv("actividades_turisticas_peru_getyourguide.csv", index=False, encoding='utf-8-sig')
